# Yolo V9


### Custom callback function for freezing of layers

In [1]:
def freeze_layer(trainer):
    model = trainer.model
    num_freeze = 10
    print(f"Freezing {num_freeze} layers")
    freeze = [f'model.{x}.' for x in range(num_freeze)]  # layers to freeze 
    for k, v in model.named_parameters(): 
        v.requires_grad = True  # train all layers 
        if any(x in k for x in freeze): 
            print(f'freezing {k}') 
            v.requires_grad = False 
    print(f"{num_freeze} layers are freezed.")

### Train model

In [2]:
from ultralytics import YOLO

# Build a YOLOv9c model from pretrained weight
model = YOLO('../pretrained-weights/yolov8s-seg.pt')

# Display model information (optional)
model.info()

# Freeze the first 10 layers (backbone) of the model
model.add_callback("on_train_start", freeze_layer)

# Train the model on the COCO8 example dataset for 100 epochs
results = model.train(data='../datasets/football-dataset-yolo/data.yaml', epochs=100, imgsz=640, amp=False)



YOLOv8s-seg summary: 261 layers, 11821056 parameters, 0 gradients, 42.9 GFLOPs
New https://pypi.org/project/ultralytics/8.2.8 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.2  Python-3.11.3 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1660 Ti, 6144MiB)
engine\trainer: task=segment, mode=train, model=../pretrained-weights/yolov8s-seg.pt, data=../datasets/football-dataset-yolo/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train7, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=False, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, 

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: tijnvandeluijtgaarden (tijntjetue). Use `wandb login --relogin` to force relogin


Freezing layer 'model.22.dfl.conv.weight'


train: Scanning C:\Users\tijnv\markiemark\repositories\Deep-Learning---Instance-Segmentation\datasets\football-dataset-yolo\train\labels.cache... 409 images, 0 backgrounds, 0 corrupt: 100%|██████████| 409/409 [00:00<?, ?it/s]

train: WARNING  C:\Users\tijnv\markiemark\repositories\Deep-Learning---Instance-Segmentation\datasets\football-dataset-yolo\train\images\209.jpg: 7 duplicate labels removed


...

                   all         51        800      0.681      0.884      0.818      0.713      0.676      0.869      0.806      0.525



100 epochs completed in 0.559 hours.
Optimizer stripped from ..\runs\segment\train7\weights\last.pt, 23.8MB
Optimizer stripped from ..\runs\segment\train7\weights\best.pt, 23.8MB

Validating ..\runs\segment\train7\weights\best.pt...
Ultralytics YOLOv8.2.2  Python-3.11.3 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1660 Ti, 6144MiB)
YOLOv8s-seg summary (fused): 195 layers, 11779987 parameters, 0 gradients, 42.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:03<00:00,  1.52s/it]
c:\Users\tijnv\AppData\Local\Programs\Python\Python311\Lib\site-packages\ultralytics\utils\plotting.py:846: RuntimeWarning: invalid value encountered in cast
  for j, box in enumerate(boxes.astype(np.int64).tolist()):
Exception in thread Thread-52 (plot_images):
OverflowError: Python int too large to convert to C long

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\tijnv\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "c:\Users\tijnv\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\tijnv\AppData\Local\Programs\Python\Python311\Lib\site-packages\ultralytics\utils\plotting.py", line 852

                   all         51        800      0.918      0.876      0.951      0.817      0.913      0.869      0.942      0.626
Speed: 0.2ms preprocess, 32.5ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to ..\runs\segment\train7


lr/pg0,▃▇███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
lr/pg1,▃▇███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
lr/pg2,▃▇███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
metrics/mAP50(B),▁█▇▆██▄█████▇█▇█▇▆███▆▇▇▆▆▇▆▇▇▆▆▆▆▆▆▅▅▅█
metrics/mAP50(M),▁▇▇▆██▄█▇███▇█▇█▇▇███▆▇▇▆▆▇▆▇▇▆▆▆▆▆▆▅▅▅█
metrics/mAP50-95(B),▁▆▆▅▆▇▄▇▇█▇███▇█▇▇███▇█▇▇▆█▇▇▇▇▆▆▇▇▇▆▆▆█
metrics/mAP50-95(M),▁▅▆▅▆█▅▇▆█▇█▇█▇▇▇▇▇▇█▆▇▇▆▆▇▆▇▇▇▆▆▆▆▇▅▆▆█
metrics/precision(B),▁▇▆▆▇▇▃▇▇██▇▇▇▆▇▆▅▇▇▇▄▆▆▅▄▅▅▄▅▄▄▄▄▄▄▃▃▃▇
metrics/precision(M),▁▇▆▅▇▇▃▇▇██▇▇▇▆▇▆▅▇▇▇▄▆▆▅▄▅▅▄▅▄▄▄▄▄▄▃▃▃▇
metrics/recall(B),▅▇▄▅▅▄█▆▄▆▄▃▄▄▂▅▂▂▄▃▄▂▂▃▄▂▁▃▂▃▃▃▃▃▂▂▅▄▅▄
metrics/recall(M),▅▆▄▄▅▄█▆▃▆▄▄▄▄▂▅▂▂▄▃▄▂▁▃▄▂▁▃▂▃▃▃▃▃▂▂▄▄▄▄


### Run inference

In [8]:
# Run inference with the YOLOv9c model on the 'bus.jpg' image

dir = '../datasets/football-players-detection/test/images/'
images = [dir + f for f in os.listdir(dir) if f.endswith('.jpg')]

results = model(images)


0: 384x640 23 persons, 877.1ms
1: 384x640 24 persons, 1 tv, 877.1ms
2: 384x640 19 persons, 1 tv, 877.1ms
3: 384x640 24 persons, 1 tv, 877.1ms
4: 384x640 19 persons, 877.1ms
5: 384x640 31 persons, 877.1ms
6: 384x640 23 persons, 1 tv, 877.1ms
7: 384x640 21 persons, 1 tv, 877.1ms
8: 384x640 23 persons, 877.1ms
9: 384x640 26 persons, 1 tv, 877.1ms
10: 384x640 24 persons, 1 tv, 877.1ms
11: 384x640 27 persons, 877.1ms
12: 384x640 19 persons, 1 tv, 877.1ms
Speed: 2.9ms preprocess, 877.1ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)


### Show inferenced image

In [7]:
# Visualize the results on the frame
annotated_frame = results[0].plot()
